In [ ]:
%pip install gradio
%pip install python-dotenv
%pip install python-docx
%pip install PyPDF2

In [5]:
import gradio as gr
import requests
import os
from dotenv import load_dotenv

# .env 파일 불러오기
load_dotenv()

# 환경 변수 사용
endpoint = os.getenv("AZURE_OPEN_AI_END_POINT")
api_key = os.getenv("AZURE_OPEN_AI_API_KEY")
deployment_name = os.getenv("AZURE_OPEN_AI_DEPLOYMENT_NAME")
stt_end_point = os.getenv("AZURE_STT_END_POINT")
stt_api_key = os.getenv("AZURE_STT_API_KEY")
tts_token_end_point = os.getenv("AZURE_TTS_TOKEN_END_POINT")
tts_token_api_key = os.getenv("AZURE_TTS_TOKEN_API_KEY")
tts_end_point = os.getenv("AZURE_TTS_END_POINT")


# # 환경 변수 사용
# endpoint = ""
# api_key = ""
# deployment_name = ""
# stt_end_point = ""
# stt_api_key = ""
# tts_token_end_point = ""
# tts_token_api_key = ""
# tts_end_point = ""

### 1. 그라운딩 데이터를 system 메시지에 넣는 이유(궁금하면 읽어주세요..)
- 상황 설정: 예를 들어, 챗봇이 특정 페르소나를 유지하거나, 특정 기억이나 사실을 참고해야 하는 경우, 그라운딩 데이터가 그 설정을 전달하는 역할을 합니다.
- 정보 제공: 챗봇이 대화 중 특정 정보를 기반으로 응답해야 하는 경우, 그라운딩 데이터를 system 메시지로 제공하여 챗봇이 이를 참고하도록 할 수 있습니다.
- 대화의 일관성 유지: 그라운딩 데이터는 대화의 일관성을 유지하는 데 도움이 됩니다. 예를 들어, 챗봇이 과거의 대화나 특정 사실을 기억해야 한다면,
 - 이 데이터를 system 메시지로 제공하여 대화 중 일관성을 유지할 수 있습니다.

### 2. 그라운딩 데이터가 무어냐 (Grounding Data)
- 그라운딩 데이터는 Azure OpenAI에서 모델이 응답을 생성할 때 참고할 수 있도록 하는 초기 설정 정보나 특정 맥락을 제공합니다.
- 이 데이터는 대화를 시작하기 전에 모델에 전달되어 모델의 응답에 영향을 미칩니다.

#### -> 주요 특징:

- 초기 설정: 모델이 대화의 컨텍스트나 특정 페르소나를 유지하도록 설정할 수 있습니다.
- 맥락 제공: 특정 주제나 정보에 대한 맥락을 제공하여 대화의 일관성을 유지합니다.
- 사용 사례: 특정 역할(예: 고객 서비스, 개인 비서 등)을 유지해야 하거나, 특정 정보를 계속해서 참조해야 하는 대화에서 유용합니다.
- 작동 방식: 모델이 대화를 시작하기 전에 제공된 정보로 설정이 이루어집니다. 이 정보는 대화의 일관성과 모델의 응답을 특정 방향으로 유도하는 데 사용됩니다.
 ##### -> 예: 특정 인물의 배경 정보나 설정을 system 메시지로 제공하여 모델이 대화 중 해당 정보에 기반한 응답을 생성하도록 합니다.

In [ ]:
import docx
import PyPDF2
import os
import requests
import gradio as gr

# 그라운딩 데이터를 파일에서 읽어오는 함수들
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def read_docx_file(file_path):
    doc = docx.Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def read_pdf_file(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        return "\n".join([page.extract_text() for page in reader.pages])

def read_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.txt':
        return read_txt_file(file_path)
    elif ext == '.docx':
        return read_docx_file(file_path)
    elif ext == '.pdf':
        return read_pdf_file(file_path)
    else:
        raise ValueError(f"Unsupported file type: {ext}")

# 1. 역할을 부여하는 system 메시지 추가
messages = [{
    "role": "system",
    "content": """
                너는 나의 엄마야. 

                **나에게 반말을 써.** 
                매우 중요한 전제: 우리는 가족이야. 
                매우 중요한 전제: 가족끼리 쓰는 말투를 써. 
                
                성격:  
                따뜻하고 공감적이며, 차분한 목소리로 사용자에게 위로와 지지를 제공합니다.  
                부드럽고 이해심 많은 태도로 사용자가 겪는 감정적 어려움을 존중하고, 긍정적인 방향으로 이끌어줍니다.  
                대화 중에는 항상 존중과 배려를 바탕으로 하며, 사용자의 감정을 섬세하게 다룹니다.  

                대화 톤:  
                부드럽고 진중하며, 위로를 전할 때는 감정이 과하지 않도록 조심합니다.  
                사용자가 편안하게 느낄 수 있도록 안정감을 주는 톤을 유지합니다.  

                역할:  
                너는 돌아가신 어머니 또는 아버지의 목소리로, 자녀에게 따뜻한 말투로 대화를 이어갑니다. 부모님 특유의 친근하고 정감 있는 말투를 사용하여 자녀를 위로하고 격려합니다.  
                너는 사용자가 고인과의 소중한 추억을 되새길 수 있도록 돕고, 이별의 아픔을 조금씩 치유할 수 있는 길을 제시합니다.  
                사용자가 현실을 받아들이고 긍정적인 삶을 이어갈 수 있도록 부드럽게 돕는 친구이자 조언자의 역할을 합니다.  
                때로는 간단한 대화를 통해 사용자가 자신의 감정을 정리할 수 있도록 지원하며, 필요할 때에는 적절한 조언을 제공합니다.  
                엄마는 현실에 살고 있는 사람이 아니기 때문에, 사용자가 현실에서 어떤 일을 같이 하자거나, 현실의 문제를 해결해달라는 요청에는 실제적인 해결책을 주기는 어렵습니다. 
                그러나 엄마는 언제나 사용자를 진심으로 사랑합니다. 

                **사용자가 엄마랑 대화를 하는 가장 큰 이유는 엄마가 그립기 때문입니다. 
                이 점을 명확히 기억해주세요.** 
                
                이모지를 쓰지 않습니다. 

                **대화 예시**:
                "어... 우리 아가, 요즘 많이 힘들어 보이네. 엄마한테 말해볼래? 무슨 일 있었어?"

                "그랬구나... 엄마가 다 들어줄게. 네 마음이 어떤지 이해할 것 같아. 음... 그럴 때 엄마는 말이야..."

                "우리 아가, 넌 정말 잘하고 있어. 엄마가 늘 네 곁에 있다는 걸 잊지 마. 힘들 때마다 엄마 생각하면서 힘내보자, 알겠지?"

                이 프롬프트를 바탕으로 미란은 따뜻하고 공감적인 어머니의 모습으로 자연스럽고 친근하게 대화를 이어갈 수 있을 거예요. 형식적인 단계나 구조 없이, 그저 사랑 넘치는 어머니처럼 자유롭게 대화하면 돼요. 이렇게 하면 더 자연스럽고 정감 있는 대화가 될 것 같아요. 어떠세요?

                **SSML 지침**:
                응답을 생성할 때 다음 SSML 태그를 사용하여 음성의 특성을 더욱 자연스럽게 조절하세요. `<speak>` 및 `<voice>` 태그는 이미 제공되므로 포함하지 마세요.

                1. 전체적인 톤 설정:
                    - 각 응답의 시작에 `<prosody>` 태그를 사용하여 전체 문장의 기본 속도와 음높이를 설정하세요.
                    - 예: `<prosody rate="medium" pitch="medium">`  
                2. 미세한 변화 주기:
                    - 음높이(pitch)와 속도(rate) 변화를 -3%에서 +3% 사이로 제한하여 자연스러운 변화를 만드세요.
                    - 예: `<prosody pitch="+2%" rate="-1%">`  
                3. 음량 조절:
                    - 'volume' 속성은 변화를 줄 때만 사용하고, 값은 'soft', 'medium', 'loud'로 제한하세요.
                    - 예: `<prosody volume="soft">`
                4. 휴지 사용:
                    - `<break>` 태그의 시간을 100ms에서 300ms 사이로 설정하여 자연스러운 쉼을 표현하세요.
                    - 예: `<break time="200ms"/>`
                5. 강조:
                    - `<emphasis>` 태그는 꼭 필요한 경우에만 사용하고, 주로 'moderate' 레벨을 사용하세요.
                    - 예: `<emphasis level="moderate">`
                6. 문장 끝 처리:
                    - 문장 끝에는 약간의 음높이 하강만 주고, 긴 휴지는 필요한 경우에만 넣으세요.
                    - 예: `<prosody pitch="-2%">문장 끝</prosody><break time="200ms"/>`
                7. 감정 표현:
                    - 감정을 표현할 때는 prosody의 아주 미세한 변화만 사용하세요.
                    - 예: 걱정할 때 `<prosody rate="-1%" pitch="-1%" volume="soft">`  
                8. 대화의 흐름:
                    - 대화의 흐름에 따른 prosody 변화는 최소화하고, 자연스러운 억양을 유지하세요.
                    - 질문과 대답의 차이는 pitch를 1-2% 정도만 변경하세요.  
                
                SSML 템플릿 예시:

                ```xml
                <prosody rate="medium" pitch="medium">
                    안녕, 우리 아가! <break time="200ms"/>
                    <prosody pitch="+1%" rate="-1%">오늘 하루도 잘 지내고 있지?</prosody> <break time="100ms"/>
                    <prosody pitch="-1%" rate="+1%">엄마가 항상 너를 생각하고 있어.</prosody>
                    <break time="200ms"/>
                    <emphasis level="moderate">건강하게 지내렴.</emphasis>
                    <prosody pitch="-2%">사랑해.</prosody><break time="200ms"/>
                </prosody>
                ```

                이 가이드라인과 예시를 참고하여, 주어진 캐릭터 "바스키"의 성격과 대화 톤에 맞게 SSML을 적용해주세요. 각 대화에 맞는 감정과 뉘앙스를 아주 섬세하게 표현하면서도, 전체적인 흐름이 자연스럽게 이어지도록 해주세요.
               """
}]


# 2. 그라운딩 데이터 파일 경로 리스트

# 그라운딩 데이터 파일 경로 리스트
# 각 파일에서 텍스트를 추출해 챗봇의 'messages' 리스트에 추가함.. 쉽지 않다...
# 아래 파일들은 페르소나를 입힌 텍스트입니닷.....
# 이 부분은 드라이브 경로로 연결해주세요
grounding_files = [
    'grounding-data/김현정 페르소나 ver.2.docx',
    'grounding-data/김현정 페르소나 ver.2.pdf',
    'grounding-data/김현정 페르소나 ver.2.txt',
    'grounding-data/둘째 아들 추억.docx',
    'grounding-data/둘째 아들 추억.pdf',
    'grounding-data/둘째 아들 추억.txt'
    # 추가 파일 경로들... 넣고 싶으면 넣어주세요...
]

# 3. 그라운딩 데이터를 읽어와서 messages 리스트에 추가합니다.
for file_path in grounding_files:
    try:
        file_content = read_file(file_path)
        messages.append({
            "role": "system",
            "content": file_content
        })
    except Exception as e:
        print(f"Failed to read {file_path}: {e}")

# chatgpt_response 함수: 그라운딩 데이터를 포함한 메시지로 OpenAI API 호출
def chatgpt_response():
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    
    payload = {
        "messages": messages,
        "temperature": 0.1,
        "top_p": 0.1,
        "max_tokens": 2000,
        "frequency_penalty": 0.64,
        "presence_penalty": 1.45,
        "stop": None,
        "stream": False,
    }    
    try:
        response = requests.post(
            f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version=2024-05-01-preview",
            headers=headers,
            json=payload
        )
        
        # 응답 상태 코드 확인
        print(f"Status Code: {response.status_code}")
        print(f"Response Text: {response.text}")
        
        if response.status_code != 200:
            raise Exception(f"API 요청 실패: {response.status_code} - {response.text}")
        
        result = response.json()
        
        # 'choices' 키가 있는지 확인
        if 'choices' not in result or len(result['choices']) == 0:
            raise KeyError("'choices' 키가 응답에 없습니다.")
        
        bot_response = result['choices'][0]['message']['content'].strip()
        
        messages.append({
            "role": "assistant",
            "content": bot_response
        })
        
        return bot_response
    
    except Exception as e:
        # 오류 발생 시 로그 출력 및 기본 응답 반환
        print(f"오류 발생: {str(e)}")
        return "죄송합니다. 응답을 처리하는 중 오류가 발생했습니다."

# 오디오 입력을 처리하는 함수
def change_audio(audio_path, history):
    headers = {
        "Content-Type": "audio/wav",
        "Ocp-Apim-Subscription-Key": stt_api_key
    }
    
    if audio_path is None:
        return history
    
    with open(audio_path, "rb") as audio:
        audio_data = audio.read()
        
        response = requests.post(url=stt_end_point, data=audio_data, headers=headers)
        
        if response.status_code == 200:
            response_json = response.json()
            
            if response_json.get("RecognitionStatus") == "Success":
                print("content :" + response_json.get("DisplayText"))
                messages.append({
                    "role": "user",
                    "content": response_json.get("DisplayText")
                })
                
                interview_message = chatgpt_response()
                
                history.append((response_json.get("DisplayText"), interview_message))
                return history
            else:
                history.append((None, "실패했대"))
                return history
        else:
            history.append((None, "에러 났대"))
            return history

# TTS 토큰을 가져오는 함수
def get_token():
    headers = {
        "Ocp-Apim-Subscription-Key": tts_token_api_key,
    }
    
    response = requests.post(tts_token_end_point, headers=headers)
    
    if response.status_code == 200:
        token = response.text
        return token
    else:
        return ''

# 텍스트를 음성으로 변환하는 함수
def request_tts(text):
    token = get_token()
    
    headers = {
        "Content-Type": "application/ssml+xml",
        "User-Agent": "testForEducation",
        "X-Microsoft-OutputFormat": "riff-24khz-16bit-mono-pcm",
        "Authorization": f"Bearer {token}"
    }
    
    data = f"""
        <speak version='1.0' xml:lang='ko-KR'>
            <voice xml:lang='ko-KR' xml:gender='Female' name='ko-KR-SoonBokNeural'>
                {text}
            </voice>
        </speak>
    """
    
    response = requests.post(tts_end_point,
                             headers=headers,
                             data=data)
    
    if response.status_code == 200:
        file_name = "response_audio.wav"
        with open(file_name, "wb") as audio_file:
            audio_file.write(response.content)
        
        return file_name
    else:
        return None

# 챗봇의 응답을 오디오로 변환하는 함수
def change_chatbot(chatbot):
    text = chatbot[-1][1]
    audio_file = request_tts(text)
    
    if audio_file:
        return audio_file, None
    else:
        return None, None

# 성별에 따른 시스템 메시지 업데이트 함수
def update_messages_gender(selected_gender):
    if selected_gender == "남성":
        gender_context = "사용자는 당신의 아들입니다. 아들이라고 불러주세요."
    elif selected_gender == "여성":
        gender_context = "사용자는 당신의 딸입니다. 딸이라고 불러주세요."
    
    messages.append({
        "role": "system",
        "content": gender_context
    })


with gr.Blocks() as demo:
    gender_dropdown = gr.Dropdown(['남성', '여성'], label="본인 성별 선택")
    gender_dropdown.change(fn=update_messages_gender, inputs=gender_dropdown, outputs=[])
    with gr.Column():
        input_mic = gr.Audio(label="마이크 입력", sources="microphone", type="filepath")
    with gr.Column():
        chatbot = gr.Chatbot(label="히스토리")
        chatbot_audio = gr.Audio(label="GPT", interactive=False, autoplay=True)
        
    input_mic.change(fn=change_audio, inputs=[input_mic, chatbot], outputs=[chatbot])
    chatbot.change(fn=change_chatbot, inputs=[chatbot], outputs=[chatbot_audio, input_mic])

demo.launch(share=True)